In [33]:
import os
import numpy as np
import pandas as pd
import pickle
import tqdm
DATA_PATH = '/mnt/data/molchanov/datasets/okko/rekko_challenge_rekko_challenge_2019'

In [2]:
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [3]:
ratings = transactions.groupby(['element_uid'])['user_uid'].count()\
                      .reset_index()\
                      .sort_values(['user_uid'], ascending=False)

In [4]:
ratings.head(10)

,element_uid,user_uid
2194,2714,68320
616,747,61272
2126,2639,61119
3059,3783,55228
1801,2245,52942
4972,6127,52168
8185,10061,48993
3173,3916,48243
7126,8771,47480
2694,3336,45646


In [6]:
ratings.to_csv(os.path.join(DATA_PATH, 'popularity_ratings.csv'))

In [7]:
ratings = pd.read_csv(os.path.join(DATA_PATH, 'popularity_ratings.csv'))
ratings.head(10)

,Unnamed: 0,element_uid,user_uid
0,2194,2714,68320
1,616,747,61272
2,2126,2639,61119
3,3059,3783,55228
4,1801,2245,52942
5,4972,6127,52168
6,8185,10061,48993
7,3173,3916,48243
8,7126,8771,47480
9,2694,3336,45646


# Popularity model

In [56]:
np.random.seed(15)

# get data
with open(f'/mnt/data/molchanov/datasets/okko/rekko_challenge_rekko_challenge_2019/transactions_grouped.pkl', 'rb') as handle:
    data = pickle.load(handle)
    data = [d for d in data if d['len'] >= 30]
    data = np.random.permutation(data)

    split_len = int(len(data)*0.7)
    train_data, test_data = data[:split_len], data[:split_len]
    
# get ratings
ratings = pd.read_csv(os.path.join(DATA_PATH, 'popularity_ratings.csv'))
ratings = ratings['element_uid'].values.tolist()

In [65]:
recals = []
N = 10
for user in tqdm.tqdm(test_data):
    seen_films = set(user['feature_arrays']['element_uid'][:-4])
    relvs_films = set(user['feature_arrays']['element_uid'][-4:])
    
    # get top 4 not seen films for user
    recs = []
    for pop_film in ratings:
        if pop_film not in seen_films:
            recs.append(pop_film)
        if len(recs) == N:
            recs = set(recs)
            break
    
    #recall
    recals.append(float(len(recs.intersection(relvs_films)))/len(relvs_films))
np.array(recals).mean()

100%|██████████| 63760/63760 [00:00<00:00, 97014.40it/s]


0.10378372020075283